In [ ]:
! pip install transformers==3.0.0

In [ ]:
import numpy as np
import torch.nn as nn
import pandas as pd
import torch
from collections import defaultdict
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

#nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')
device = torch.device("cuda")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# import os

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))

# tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Project_2')

In [ ]:
df = pd.read_csv("/content/drive/My Drive/Project_2/train_df.csv")
df_ = pd.read_csv("/content/drive/My Drive/Project_2/valid_df.csv")

In [ ]:
def pp(df1):
  #df['mod_text'] = df['text'].replace('\d+', '')
  df1['mod_text'] = df1['text'].apply(lambda x: x[2:-2]) #Removing brackets
  df1['mod_text'] = df1['mod_text'].str.replace('#','') #\S -> Non space character being removed.
  df1['mod_text'] = df1['mod_text'].str.replace('@(\S+) ','') #Removing hastags and username and tags
  df1['mod_text'] = df1['mod_text'].str.replace('https:\S+','') #Removing url
  df1['mod_text'] = df1['mod_text'].str.replace(r'\s+|\\n', ' ', regex=True) #Removing next line
  df1['mod_text'] = df1['mod_text'].str.replace('[^a-zA-Z0-9]',' ')
  #df1['final'] = df1['mod_text'].str.rstrip('\n')
  #df1['final'] = df1['final'].str.replace(' +', '')
  #df['mod_text'] = df['mod_text'].apply(lambda x: x[2:-2])
  tokenized_doc = df1['mod_text'].apply(lambda x: x.split())
  tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
  detokenized_doc = [] 
  for i in range(len(df1)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 
  df1['final'] = detokenized_doc

In [ ]:
pp(df)
pp(df_)
df.final=df.final.apply(str)
df_.final=df_.final.apply(str)

In [ ]:
df_train, df_val = train_test_split( df , 
random_state=20, test_size=0.3, stratify=df['label'])

In [ ]:
MAX_LEN = 65

In [ ]:
class Covid_Occup(Dataset):

    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = str(self.texts[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        truncation = True,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'text_text': text,
        'input_ids': input_ids.flatten(),
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = Covid_Occup(
    texts=df.final.to_numpy(),
    targets=df.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
  )

In [ ]:
from transformers import XLNetTokenizer, XLNetModel

PRE_TRAINED_MODEL_NAME = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
BATCH_SIZE = 6
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
#test_data_loader = create_data_loader(df_, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
from transformers import XLNetForSequenceClassification
#with tpu_strategy.scope(): 
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2)
model = model.to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
EPOCHS = 10

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)
#optimizer = AdamW(model.parameters(), lr = 3e-4)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

In [ ]:
from sklearn import metrics
def train_epoch(model, data_loader, optimizer, device, #scheduler, 
                n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        targets = d['targets'].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())
        
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)



In [ ]:
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
    #prec = 0

    pred = np.array([])

    with torch.no_grad():
        for d in data_loader:
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            targets = d['targets'].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)
            #precision = metrics.precision_score(targets,prediction)
            #print(prediction)
            pred = np.concatenate((pred, prediction))

            acc += accuracy
            #prec += precision
            losses.append(loss.item())
            counter += 1

    return acc / counter, np.mean(losses), pred



In [ ]:
#len(train_text)

In [ ]:
data = next(iter(train_data_loader))
#data.keys()

In [ ]:
 input_ids = data['input_ids'].to(device)
 attention_mask = data['attention_mask'].to(device)
 targets = data['targets']#.to(device)
 print(input_ids.shape) # batch size x seq length
 print(attention_mask.shape)

torch.Size([6, 65])
torch.Size([6, 65])


In [ ]:
# outputs = model(input_ids.reshape(BATCH_SIZE,MAX_LEN), token_type_ids=None, attention_mask=attention_mask, labels=targets)
# #outputs
# _, prediction = torch.max(outputs[1], dim=1)
# prediction = prediction.cpu().detach().numpy()
# print(prediction)



In [ ]:
#type(outputs[0])

In [ ]:
#%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(model, train_data_loader, optimizer, device, #scheduler, 
                                        len(df_train))

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss, _ = eval_model(model,val_data_loader, device, len(df_val))

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), '/content/drive/My Drive/Project_2/xlnet_model.bin')
        best_accuracy = val_acc



Epoch 1/10
----------
Train loss 0.4789730848119195 Train accuracy 0.8335714285714277
Val loss 0.2915901771001518 Val accuracy 0.925555555555556

Epoch 2/10
----------
Train loss 0.31883435994203735 Train accuracy 0.9304761904761915
Val loss 0.26186799774101627 Val accuracy 0.9344444444444452

Epoch 3/10
----------
Train loss 0.2056337396111173 Train accuracy 0.9595238095238103
Val loss 0.3486472879024223 Val accuracy 0.9366666666666674

Epoch 4/10
----------
Train loss 0.15149781900682552 Train accuracy 0.9709523809523817
Val loss 0.37088444287733485 Val accuracy 0.9322222222222226

Epoch 5/10
----------
Train loss 0.10051445204506827 Train accuracy 0.9819047619047627
Val loss 0.4396826852244946 Val accuracy 0.923888888888889

Epoch 6/10
----------
Train loss 0.07317570027026314 Train accuracy 0.9880952380952387
Val loss 0.35085222459941484 Val accuracy 0.9466666666666672

Epoch 7/10
----------
Train loss 0.06276898878231545 Train accuracy 0.9900000000000004
Val loss 0.384557201949161

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/Project_2/xlnet_model.bin'))

<All keys matched successfully>

In [ ]:
model = model.to(device)

In [ ]:
# data = next(iter(test_data_loader))
# #data.keys()
# input_ids = data['input_ids'].to(device)
# attention_mask = data['attention_mask'].to(device)
# targets = data['targets'].to(device)
# print(input_ids.shape)
# print(attention_mask.shape)

In [ ]:
# outputs = model(input_ids.reshape(BATCH_SIZE,MAX_LEN), token_type_ids=None, attention_mask=attention_mask, labels=targets)
# outputs

In [ ]:
test_data_loader = create_data_loader(df_, tokenizer, MAX_LEN, BATCH_SIZE)
test_acc, test_loss, pred = eval_model(model,test_data_loader,device,len(df_))

print('Test Accuracy :', test_acc)
print('Test Loss :', test_loss)
#print('Test Precesion :', test_precision)

Test Accuracy : 0.9361277445109781
Test Loss : 0.4096929946908041


In [ ]:
confusion_matrix(df_.label,pred)
#pred

array([[1463,   60],
       [  68,  409]])

In [ ]:
test = pd.DataFrame()
test['index'] = df_['tweet_id']
test['XL_Net_label'] = pred


In [ ]:
from google.colab import files
test.to_csv('test_XLNET.csv')
#files.download('test_XLNET.csv')
!cp test_XLNET.csv "/content/drive/My Drive/Project_2/test_XLNET.csv"